In [5]:
import os
import json
import pandas as pd

In [6]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    print(data)
    return data

In [7]:
eval_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/evaluation/medivdqa"
if not os.path.exists(eval_folder):
    os.makedirs(eval_folder)

vid_json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/medvidqa_finetuned_on_full"
if not os.path.exists(vid_json_folder):
    os.makedirs(vid_json_folder)

val_result_json_path = f"{vid_json_folder}/all_data_val_moment_retrieval_BEST_without_audio.json"
test_result_json_path = f"{vid_json_folder}/all_data_test_moment_retrieval_BEST_without_audio.json"
val_result_wa_json_path = f"{vid_json_folder}/all_data_val_moment_retrieval_BEST_with_audio.json"
test_result_wa_json_path = f"{vid_json_folder}/all_data_test_moment_retrieval_BEST_with_audio.json"

In [8]:
val = load_jsonl(f'{vid_json_folder}/all_data_val.json')
test = load_jsonl(f'{vid_json_folder}/all_data_test.json')

test_result = load_result_json(test_result_json_path)
val_result = load_result_json(val_result_json_path)
test_wa_result = load_result_json(test_result_wa_json_path)
val_wa_result = load_result_json(val_result_wa_json_path)

{'How to perform chin tucks to treat neck pain?': {'h5MvX50zTLM.mp4': {'bounds': [84, 78], 'video_duration': 186}}, 'How to perform scapular retractions to treat neck pain?': {'h5MvX50zTLM.mp4': {'bounds': [84, 129], 'video_duration': 186}}, 'How to perform corner stretches to treat neck pain?': {'h5MvX50zTLM.mp4': {'bounds': [84, 129], 'video_duration': 186}}, 'How can I do hamstring stretches for getting rid of lower back pain?': {'Se3kf5X4PCc.mp4': {'bounds': [71, 71], 'video_duration': 123}}, 'How can I do stretching of piriformis muscles to get rid of lower back pain?': {'Se3kf5X4PCc.mp4': {'bounds': [71, 53], 'video_duration': 123}}, 'How to do knee to opp. shoulder to loosen lower back and hip tightness?': {'Se3kf5X4PCc.mp4': {'bounds': [73, 53], 'video_duration': 123}}, 'How do you put on a clavicle brace?': {'UZqktEPlxTU.mp4': {'bounds': [54, 87], 'video_duration': 118}}, 'How to assess the supraclavicular lymph nodes': {'_N89T_Yqu68.mp4': {'bounds': [107, 115], 'video_duratio

In [9]:
def compute_iou(interval_1, interval_2):
    start_i, end_i = interval_1[0], interval_1[1]
    start, end = interval_2[0], interval_2[1]
    intersection = max(0, min(end, end_i) - max(start, start_i))
    union = min(max(end, end_i) - min(start, start_i), end-start + end_i-start_i)
    iou = float(intersection) / (union + 1e-8)
    return iou

In [10]:
iou_thresh_3, iou_thresh_5, iou_thresh_7, total, queries_cnt = 0, 0, 0, 0, 0
test_woa_under_thresh, test_wa_under_thresh, val_woa_under_thresh, val_wa_under_thresh = {}, {}, {}, {}
val_wa_under_thresh['low'], val_wa_under_thresh['high'] = {}, {}
val_woa_under_thresh['low'], val_woa_under_thresh['high'] = {}, {}
test_wa_under_thresh['low'], test_wa_under_thresh['high'] = {}, {}
test_woa_under_thresh['low'], test_woa_under_thresh['high'] = {}, {}


for key, value in test[0].items():
    if key in test_result:
        vid = next(iter(value))
        #print(vid)
        iou = compute_iou(value[vid]['bounds'], test_result[key][vid]['bounds'])
        if test_result[key][vid]['bounds'][0] > test_result[key][vid]['bounds'][1]:
            print("ERROR")
        if iou < 0.3:
            if 'queries' not in test_woa_under_thresh['low']:
                test_woa_under_thresh['low']['queries'] = []
                test_woa_under_thresh['low']['videos'] = []
                test_woa_under_thresh['low']['iou'] = []
            else:
                test_woa_under_thresh['low']['queries'].append(key)
                test_woa_under_thresh['low']['videos'].append(vid)
                test_woa_under_thresh['low']['iou'].append(iou)
            if 'duration' not in test_woa_under_thresh['low']:
                test_woa_under_thresh['low']['duration'] = []
            else:
                test_woa_under_thresh['low']['duration'].append([value[vid]['bounds'], test_result[key][vid]['bounds']])
        if iou >= 0.3:
            iou_thresh_3 += 1 
        if iou >= 0.5:
            iou_thresh_5 += 1 
        if iou >= 0.7:
            print(key)
            iou_thresh_7 += 1 
            if 'queries' not in test_woa_under_thresh['high']:
                test_woa_under_thresh['high']['queries'] = []
                test_woa_under_thresh['high']['videos'] = []
                test_woa_under_thresh['high']['iou'] = []
            else:
                test_woa_under_thresh['high']['queries'].append(key)
                test_woa_under_thresh['high']['videos'].append(vid)
                test_woa_under_thresh['high']['iou'].append(iou)
            if 'duration' not in test_woa_under_thresh['high']:
                test_woa_under_thresh['high']['duration'] = []
            else:
                test_woa_under_thresh['high']['duration'].append([value[vid]['bounds'], test_result[key][vid]['bounds']])
        total += 1
    else:
        print("Not found")
    queries_cnt += 1
print(f"TEST without audio IoU=0.3: {iou_thresh_3/(total) * 100}\nIoU=0.5: {iou_thresh_5/(total) * 100}\nIoU=0.7: {iou_thresh_7/(total) * 100}\ntotal queries:{(total)}/{queries_cnt}")

iou_thresh_3, iou_thresh_5, iou_thresh_7, total, queries_cnt = 0, 0, 0, 0, 0
for key, value in val[0].items():
    if key in val_result:
        vid = next(iter(value))
        #print(vid)
        iou = compute_iou(value[vid]['bounds'], val_result[key][vid]['bounds'])
        if val_result[key][vid]['bounds'][0] > val_result[key][vid]['bounds'][1]:
            print("ERROR")
        if iou < 0.3:
            if 'queries' not in val_woa_under_thresh['low']:
                val_woa_under_thresh['low']['queries'] = []
                val_woa_under_thresh['low']['videos'] = []
                val_woa_under_thresh['low']['iou'] = []
            else:
                val_woa_under_thresh['low']['queries'].append(key)
                val_woa_under_thresh['low']['videos'].append(vid)
                val_woa_under_thresh['low']['iou'].append(iou)
            if 'duration' not in val_woa_under_thresh['low']:
                val_woa_under_thresh['low']['duration'] = []
            else:
                val_woa_under_thresh['low']['duration'].append([value[vid]['bounds'], val_result[key][vid]['bounds']])
        if iou >= 0.3:
            iou_thresh_3 += 1 
        if iou >= 0.5:
            iou_thresh_5 += 1 
        if iou >= 0.7:
            print(key)
            iou_thresh_7 += 1 
            if 'queries' not in val_woa_under_thresh['high']:
                val_woa_under_thresh['high']['queries'] = []
                val_woa_under_thresh['high']['videos'] = []
                val_woa_under_thresh['high']['iou'] = []
            else:
                val_woa_under_thresh['high']['queries'].append(key)
                val_woa_under_thresh['high']['videos'].append(vid)
                val_woa_under_thresh['high']['iou'].append(iou)
            if 'duration' not in val_woa_under_thresh['high']:
                val_woa_under_thresh['high']['duration'] = []
            else:
                val_woa_under_thresh['high']['duration'].append([value[vid]['bounds'], val_result[key][vid]['bounds']])
        total += 1
    else:
        print(f" not found")
    queries_cnt += 1
print(f"\nVAL without audio IoU=0.3: {iou_thresh_3/(total) * 100}\nIoU=0.5: {iou_thresh_5/(total) * 100}\nIoU=0.7: {iou_thresh_7/(total) * 100}\ntotal queries:{(total)}/{queries_cnt}")

iou_thresh_3, iou_thresh_5, iou_thresh_7, total, queries_cnt = 0, 0, 0, 0, 0
for key, value in test[0].items():
    if key in test_wa_result:
        vid = next(iter(value))
        #print(vid)
        iou = compute_iou(value[vid]['bounds'], test_wa_result[key][vid]['bounds'])
        if test_wa_result[key][vid]['bounds'][0] > test_wa_result[key][vid]['bounds'][1]:
            print("ERROR")
        if iou < 0.3:
            if 'queries' not in test_wa_under_thresh['low']:
                test_wa_under_thresh['low']['queries'] = []
                test_wa_under_thresh['low']['videos'] = []
                test_wa_under_thresh['low']['iou']= [] 
            else:
                test_wa_under_thresh['low']['queries'].append(key)
                test_wa_under_thresh['low']['videos'].append(vid)
                test_wa_under_thresh['low']['iou'].append(iou)
            if 'duration' not in test_wa_under_thresh['low']:
                test_wa_under_thresh['low']['duration'] = []
            else:
                test_wa_under_thresh['low']['duration'].append([value[vid]['bounds'], test_wa_result[key][vid]['bounds']])
        if iou >= 0.3:
            iou_thresh_3 += 1 
        if iou >= 0.5:
            iou_thresh_5 += 1 
        if iou >= 0.7:
            print(key)
            iou_thresh_7 += 1 
            if 'queries' not in test_wa_under_thresh['high']:
                test_wa_under_thresh['high']['queries'] = []
                test_wa_under_thresh['high']['videos'] = []
                test_wa_under_thresh['high']['iou'] = []
            else:
                test_wa_under_thresh['high']['queries'].append(key)
                test_wa_under_thresh['high']['videos'].append(vid)
                test_wa_under_thresh['high']['iou'].append(iou)
            if 'duration' not in test_wa_under_thresh['high']:
                test_wa_under_thresh['high']['duration'] = []
            else:
                test_wa_under_thresh['high']['duration'].append([value[vid]['bounds'], test_wa_result[key][vid]['bounds']])
        total += 1
    else:
        print("Not found")
    queries_cnt += 1
print(f"\nTEST with audio IoU=0.3: {iou_thresh_3/(total) * 100}\nIoU=0.5: {iou_thresh_5/(total) * 100}\nIoU=0.7: {iou_thresh_7/(total) * 100}\ntotal queries:{(total)}/{queries_cnt}")

iou_thresh_3, iou_thresh_5, iou_thresh_7, total, queries_cnt = 0, 0, 0, 0, 0
for key, value in val[0].items():
    if key in val_wa_result:
        vid = next(iter(value))
        #print(vid)
        iou = compute_iou(value[vid]['bounds'], val_wa_result[key][vid]['bounds'])
        if val_wa_result[key][vid]['bounds'][0] > val_wa_result[key][vid]['bounds'][1]:
            print("ERROR", iou)
        if iou < 0.3:
            if 'queries' not in val_wa_under_thresh['low']:
                val_wa_under_thresh['low']['queries'] = []
                val_wa_under_thresh['low']['videos'] = []
                val_wa_under_thresh['low']['iou'] = []
            else:
                val_wa_under_thresh['low']['queries'].append(key)
                val_wa_under_thresh['low']['videos'].append(vid)
                val_wa_under_thresh['low']['iou'].append(iou)
            if 'duration' not in val_wa_under_thresh['low']:
                val_wa_under_thresh['low']['duration'] = []
            else:
                val_wa_under_thresh['low']['duration'].append([value[vid]['bounds'], val_wa_result[key][vid]['bounds']])
        if iou >= 0.3:
            iou_thresh_3 += 1 
        if iou >= 0.5:
            iou_thresh_5 += 1 
        if iou >= 0.7:
            print(key)
            iou_thresh_7 += 1 
            if 'queries' not in val_wa_under_thresh['high']:
                val_wa_under_thresh['high']['queries'] = []
                val_wa_under_thresh['high']['videos'] = []
                val_wa_under_thresh['high']['iou'] = []
            else:
                val_wa_under_thresh['high']['queries'].append(key)
                val_wa_under_thresh['high']['videos'].append(vid)
                val_wa_under_thresh['high']['iou'].append(iou)
            if 'duration' not in val_wa_under_thresh['high']:
                val_wa_under_thresh['high']['duration'] = []
            else:
                val_wa_under_thresh['high']['duration'].append([value[vid]['bounds'], val_wa_result[key][vid]['bounds']])
        total += 1
    else:
        print(f" not found")
    queries_cnt += 1
print(f"\nVAL with audio IoU=0.3: {iou_thresh_3/(total) * 100}\nIoU=0.5: {iou_thresh_5/(total) * 100}\nIoU=0.7: {iou_thresh_7/(total) * 100}\ntotal queries:{(total)}/{queries_cnt}")


ERROR
How to perform scapular retractions to treat neck pain?
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
How to treat a herniated disc with a bridge exercise?
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
TEST without audio IoU=0.3: 8.108108108108109
IoU=0.5: 2.027027027027027
IoU=0.7: 1.3513513513513513
total queries:148/148
ERROR
ERROR
ERROR
How to inspect and palpation of the biceps muscles?
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR

VAL without audio IoU=0.3: 6.382978723404255
IoU=0.5: 2.127659574468085
IoU=0.7: 0.7092198581560284
total queries:141/141
ERROR
ERROR
How to wash your skin after radiation therapy?
How to do a child's pose full back stretching to help with back tightness stiffness?
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
How to relieve neck pain with chin tucks? 
ERROR
ERROR
ERROR
ERROR
ERROR

TE

In [10]:
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
#plot functions
def group_lengths(lengths):
    grouped = defaultdict(int)
    for length in lengths:
        # Grouping by intervals of 5
        interval = 5 * (length // 5)
        grouped[interval] += 1
    return grouped
def plot_bars(x1, x2, name1, name2, title, xlabel, ylabel):
    # Count the lengths of the strings in both lists
    lengths1 = [len(s) for s in x1]
    grouped_lengths1 = group_lengths(lengths1)

    lengths2 = [len(s) for s in x2]
    grouped_lengths2 = group_lengths(lengths2)

    # Get a combined set of all unique length intervals
    all_intervals = sorted(set(grouped_lengths1.keys()).union(set(grouped_lengths2.keys())))

    # Prepare data for the bar plot
    instances1 = [grouped_lengths1[interval] for interval in all_intervals]
    instances2 = [grouped_lengths2[interval] for interval in all_intervals]


    # Set up the positions for the bars
    bar_width = 0.35
    index = np.arange(len(all_intervals))

    # Plotting the bar plot
    fig, ax = plt.subplots()
    bar1 = ax.bar(index, instances1, bar_width, label=name1)
    bar2 = ax.bar(index + bar_width, instances2, bar_width, label=name2)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(all_intervals)
    ax.legend()

    plt.show()

In [77]:

#compare audio and without audio result
common_queries_test = {}
common_queries_test['low'], common_queries_test['high'] = {}, {}
common_queries_test['low']['queries'], common_queries_test['high']['queries'] = [], []
common_queries_test['low']['duration'], common_queries_test['high']['duration'] = [], []
common_queries_test['low']['videos'], common_queries_test['high']['videos'] = [], []
test_woa_excl, test_wa_excl = [], []
print("Test set without audio did not perform well on following queries:\n")
for idx, q in enumerate(test_woa_under_thresh['low']['queries']):
    if q in test_wa_under_thresh['low']['queries']:
        common_queries_test['low']['queries'].append(q)
        common_queries_test['low']['videos'].append(test_woa_under_thresh['low']['videos'][idx])
        common_queries_test['low']['duration'].append(test_woa_under_thresh['low']['duration'][idx])
    else:
        print(q, test_woa_under_thresh['low']['videos'][idx])
        test_woa_excl.append(test_woa_under_thresh['low']['duration'][idx])
        if "lymph" in q:
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["submental lymph"])
            
        elif "blood circulation" in q:
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["improve blood circulation", "downward dog pose"])
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["blood circulation", "dog pose"])
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["circulation", "dog pose"])
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["blood", "dog pose"])
        elif "rotator" in q:
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["grueling rotator cuff", "broomstick"])
            lf_subtitle(test_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["rotator cuff", "Broomstick"])
        print("\n\nground truth: ", test_woa_under_thresh['low']['duration'][idx][0], 
              "\npredicted (w/o audio) bounds: ", test_woa_under_thresh['low']['duration'][idx][1],
              "\npredicted (w/ audio) bounds: ", test_wa_result[q][test_woa_under_thresh['low']['videos'][idx]]['bounds'], 
                "\npredicted (w/o audio) iou: ",  test_woa_under_thresh['low']['iou'][idx], 
                "\npredicted (w/ audio) iou: ",  compute_iou(test_woa_under_thresh['low']['duration'][idx][0],test_wa_result[q][test_woa_under_thresh['low']['videos'][idx]]['bounds']))
#print("duration:", test_woa_excl)

print("\n\nTest set with audio did not perform well on following queries:\n")
for idx, q in enumerate(test_wa_under_thresh['low']['queries']):
    if q not in test_woa_under_thresh['low']['queries']:
        print(q, test_wa_under_thresh['low']['videos'][idx])
        test_wa_excl.append(test_wa_under_thresh['low']['duration'][idx])
        if "scapular" in q:
            lf_subtitle(test_wa_under_thresh['low']['videos'][idx].strip(".mp4"), ["scapular", "neck pain"])
            print(test_wa_under_thresh['low']['duration'][idx], test_wa_under_thresh['low']['iou'][idx])
        else:
            lf_subtitle(test_wa_under_thresh['low']['videos'][idx].strip(".mp4"), ["corner stretches", "neck pain"])
            print(test_wa_under_thresh['low']['duration'][idx], test_wa_under_thresh['low']['iou'][idx])
        print("\n\nground truth: ", test_wa_under_thresh['low']['duration'][idx][0], 
              "\npredicted (w/ audio) bounds: ", test_wa_under_thresh['low']['duration'][idx][1],
              "\npredicted (w/o audio) bounds: ", test_result[q][test_wa_under_thresh['low']['videos'][idx]]['bounds'], 
                "\npredicted (w/ audio) iou: ",  test_wa_under_thresh['low']['iou'][idx], 
                "\npredicted (w/o audio) iou: ",  compute_iou(test_wa_under_thresh['low']['duration'][idx][0],test_result[q][test_wa_under_thresh['low']['videos'][idx]]['bounds']))



# print("duration:", test_wa_excl)

# print("common queries with low performance\n")
# [print(q) for q in common_queries_test['low']['queries']]
# print(len(common_queries_test['low']['queries']))

# plot_bars(test_woa_under_thresh['low']['queries'], test_woa_under_thresh['high']['queries'], "low_iou", "high_iou", "Query length distribution for test set(without audio)", "query_length", "count")
# plot_bars(test_wa_under_thresh['low']['queries'], test_wa_under_thresh['high']['queries'], "low_iou", "high_iou", "Query length distribution for test set (with audio)", "query_length", "count")

Test set without audio did not perform well on following queries:

How to examine the submental lymph nodes? _N89T_Yqu68.mp4
/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/medvidqa/ASR/_N89T_Yqu68.srt
{'00': {'speaker': [], 'start_time': 0, 'end_time': 6, 'subtitle': 'The lymph node examination is part of the general examination and, as with any procedure'}, '06': {'speaker': [], 'start_time': 6, 'end_time': 10, 'subtitle': 'on a patient, you must make sure you wash your hands, introduce yourself to the patient'}, '010': {'speaker': [], 'start_time': 10, 'end_time': 15, 'subtitle': 'and take adequate consent. This is particularly important with an examination of the lymph'}, '015': {'speaker': [], 'start_time': 15, 'end_time': 21, 'subtitle': 'nodes as the majority of the examination is done from behind the patient, therefore it'}, '021': {'speaker': [], 'start_time': 21, 'end_time': 25, 'subtitle': 'is vital that the patient knows what you are about to do and where you wish placing

In [46]:

#compare audio and without audio result
common_queries_val = {}
common_queries_val['low'], common_queries_val['high'] = {}, {}
common_queries_val['low']['queries'], common_queries_val['high']['queries'] = [], []
common_queries_val['low']['duration'], common_queries_val['high']['duration'] = [], []
common_queries_val['low']['videos'], common_queries_val['high']['videos'] = [], []
val_woa_excl, val_wa_excl = [], []
print("Val set without audio did not perform well on following queries:\n")
for idx, q in enumerate(val_woa_under_thresh['low']['queries']):
    if q in val_wa_under_thresh['low']['queries']:
        common_queries_val['low']['queries'].append(q)
        common_queries_val['low']['videos'].append(val_woa_under_thresh['low']['videos'][idx])
        common_queries_val['low']['duration'].append(val_woa_under_thresh['low']['duration'][idx])
    else:
        print(q, val_woa_under_thresh['low']['videos'][idx])
        val_woa_excl.append(val_woa_under_thresh['low']['duration'][idx])
        lf_subtitle(val_woa_under_thresh['low']['videos'][idx].strip(".mp4"), ["self-massage", "self massage"])
        print(val_woa_under_thresh['low']['duration'][idx])
# print("duration:", val_woa_excl)
print("\n\Val set with audio did not perform well on following queries:\n")
for idx, q in enumerate(val_wa_under_thresh['low']['queries']):
    if q not in val_woa_under_thresh['low']['queries']:
        print(q, val_wa_under_thresh['low']['videos'][idx])
        val_wa_excl.append(val_wa_under_thresh['low']['duration'][idx])
        if "stroke" in q:
            lf_subtitle(val_wa_under_thresh['low']['videos'][idx].strip(".mp4"), ["key signs", "stroke"])
            print(val_wa_under_thresh['low']['duration'][idx])
        else: 
            lf_subtitle(val_wa_under_thresh['low']['videos'][idx].strip(".mp4"), ["defibrillation", "external paddles"])
            print(val_wa_under_thresh['low']['duration'][idx])


# print("duration:", val_wa_excl)

# print("common queries with low performance\n")
# [print(q) for q in common_queries_val['low']['queries']]
# print(len(common_queries_val['low']['queries']))

#plot_bars(val_woa_under_thresh['low']['queries'], val_woa_under_thresh['high']['queries'], "low_iou", "high_iou", "Query length distribution for validation set (without audio)", "query_length", "count")
#plot_bars(val_wa_under_thresh['low']['queries'], val_wa_under_thresh['high']['queries'], "low_iou", "high_iou", "Query length distribution validation set (with audio)", "query_length", "count")

Val set without audio did not perform well on following queries:

How should i do self-massage to relieve foot pain?  ko-uitcuY8Q.mp4
/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/medvidqa/ASR/ko-uitcuY8Q.srt
43 46 In the first part, we're going to do a self massage, known as muscle
91 95 towards the toes, just applying enough pressure in there to get a self-massage.
[[45, 207], [2, 546]]

\Val set with audio did not perform well on following queries:

How to do manual defibrillation using external paddles? Dubih8EzBJQ.mp4
/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/medvidqa/ASR/Dubih8EzBJQ.srt
0 14 With manual defibrillation, you control the defibrillation process.
14 19 The Heart Start Intrepid lets you perform manual defibrillation using multifunction
19 24 electrode pads, external paddles, and internal paddles.
24 30 Here, we'll demonstrate manual defibrillation using external paddles.
30 35 After assessing your patient and determining that defibrillation is necessary,
81

In [76]:
import pysrt
import re
def lf_subtitle(video_name, keywords, subtitle_root_path = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/medvidqa/ASR"):
    '''
    Labeling function to find keywords in a video subtitle

    args:
        transcript (list of tuples) : timestamped transcript as a list of timestamp and transcribed string pairs (as tuples)
        keywords (list of str) : list of keywords

    return:
        list of timestamps
    '''
    T = []
    subtitle_path = os.path.join(subtitle_root_path, video_name + ".srt")
    print(subtitle_path)
    subtitle = pysrt.open(subtitle_path)
    speaker = ""
    subtitle_dict = {} # key is start time
    for sub in subtitle:
        pattern = r'\(([^:]+):\)'  # This pattern captures everything between ( and :)
        name = re.findall(pattern, sub.text)
        #print(sub.text)
        if name != "":
            speaker = name
        else:
            speaker = None
        s_min, s_sec, e_min, e_sec = sub.start.minutes, sub.start.seconds, sub.end.minutes, sub.end.seconds
        subtitle_dict[str(s_min) + str(s_sec)] = {'speaker': speaker, 
                                                'start_time' : s_min * 60 + s_sec,
                                                'end_time' : e_min * 60 + e_sec,
                                                'subtitle' : sub.text}

    
    print(subtitle_dict)

    for keyword in keywords:
        keyword = keyword.replace(".", "").replace(",", "").replace("_", "").replace("-", " ")
    #print(keywords)
    keyword_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\b', flags=re.IGNORECASE)   
    for k, v in subtitle_dict.items():
        if keyword_pattern.search(v['subtitle']):
            T.append([v['start_time'], v['end_time']])
            print(v['start_time'], v['end_time'], v['subtitle'])


    return T
